# MIS 6341 : AML Project 1

### Group 23 : Tejas Kulkarni , Kaustubh Sharma

## Regression Task

## Interest rate prediction using customer loan data

Number of rows: *2200* ; Number of columns : *15* 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('loan_data_cleaned.csv')

In [3]:
x = df.drop('Interest.Rate',axis=1)
y = df['Interest.Rate']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
x_train_og,x_test_og,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
#scaler = MinMaxScaler()
scaler = StandardScaler()

In [7]:
x_train = scaler.fit_transform(x_train_og)
x_test = scaler.transform(x_test_og)

### 1. Apply any two models with bagging and any two models with pasting.

#### Applying Ridge with bagging

In [8]:
from sklearn.linear_model import Lasso,Ridge
from sklearn.model_selection import GridSearchCV

In [9]:
lambdas=np.linspace(13,17,100)
model=Ridge(fit_intercept=True,random_state=0)
params={'alpha':lambdas}
grid_search_ridge = GridSearchCV(model,params,cv=5)
grid_search_ridge.fit(x_train, y_train)
print("Best parameters: {}".format(grid_search_ridge.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search_ridge.best_score_))

Best parameters: {'alpha': 15.141414141414142}
Best cross-validation score: 0.7499


In [10]:
from sklearn.ensemble import BaggingRegressor

In [11]:
paramgridls = {'n_estimators':[200, 300, 400, 500], 
              'max_samples':[0.1, 0.2, 0.3, 0.4, 0.5], 
              'max_features':[0.1, 0.2, 0.3, 0.5]}

baggrid = BaggingRegressor(Ridge(alpha = grid_search_ridge.best_params_['alpha'],random_state=0))
baggrid = GridSearchCV(baggrid, param_grid=paramgridls , cv = 5, n_jobs = -1)
baggrid.fit(x_train, y_train)


print(' Best Parameters:',baggrid.best_params_)
#print(' Best Score:',baggrid.best_score_)


 Best Parameters: {'max_features': 0.5, 'max_samples': 0.4, 'n_estimators': 300}


#### Bagging

In [12]:
model = Ridge(fit_intercept=True,alpha=grid_search_ridge.best_params_['alpha'])
bag_reg = BaggingRegressor(model, n_estimators=baggrid.best_params_['n_estimators'],
                           max_samples=baggrid.best_params_['max_samples'],max_features=baggrid.best_params_['max_features'],
                           bootstrap=True, random_state=0, oob_score = True)

bag_reg.fit(x_train, y_train)
y_pred = bag_reg.predict(x_test)

In [13]:
print('Train score: %.2f'%bag_reg.score(x_train, y_train))
print('Test score: %.2f'%bag_reg.score(x_test, y_test))
print('Out-of-bag score: %.2f'%bag_reg.oob_score_)

Train score: 0.57
Test score: 0.58
Out-of-bag score: 0.56


#### Pasting

In [14]:
model = Ridge(fit_intercept=True,alpha=grid_search_ridge.best_params_['alpha'])
bag_reg = BaggingRegressor(model, n_estimators=baggrid.best_params_['n_estimators'],
                           max_samples=baggrid.best_params_['max_samples'],max_features=baggrid.best_params_['max_features'],
                           bootstrap=False, random_state=0)

bag_reg.fit(x_train, y_train)
y_pred = bag_reg.predict(x_test)

In [15]:
print('Train score: %.2f'%bag_reg.score(x_train, y_train))
print('Test score: %.2f'%bag_reg.score(x_test, y_test))

Train score: 0.57
Test score: 0.58


#### Applying Lasso with bagging

In [16]:
lamdas = np.linspace(0.1,0.001,100)
model=Lasso()
params={'alpha':lamdas}
grid_search_lasso = GridSearchCV(model,params,cv=5)
grid_search_lasso.fit(x_train, y_train)
print("Best parameters: {}".format(grid_search_lasso.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search_lasso.best_score_))

Best parameters: {'alpha': 0.032}
Best cross-validation score: 0.7526


In [17]:
paramgridls = {'n_estimators':[200, 300, 400, 500], 
              'max_samples':[0.1, 0.2, 0.3, 0.4, 0.5], 
              'max_features':[0.1, 0.2, 0.3, 0.5]}

baggrid = BaggingRegressor(Lasso(alpha = grid_search_lasso.best_params_['alpha'],random_state=0))
baggrid = GridSearchCV(baggrid, param_grid=paramgridls , cv = 5, n_jobs = -1)
baggrid.fit(x_train, y_train)


print(' Best Parameters:',baggrid.best_params_)
#print(' Best Score:',baggrid.best_score_)


 Best Parameters: {'max_features': 0.5, 'max_samples': 0.4, 'n_estimators': 200}


#### Bagging

In [18]:
model = Lasso(fit_intercept=True,alpha=grid_search_lasso.best_params_['alpha'],random_state=0)
bag_reg = BaggingRegressor(model, n_estimators=baggrid.best_params_['n_estimators'],
                           max_samples=baggrid.best_params_['max_samples'],max_features=baggrid.best_params_['max_features'],
                           bootstrap=True, random_state=0, oob_score = True)

bag_reg.fit(x_train, y_train)
y_pred = bag_reg.predict(x_test)

In [19]:
print('Train score: %.2f'%bag_reg.score(x_train, y_train))
print('Test score: %.2f'%bag_reg.score(x_test, y_test))
print('Out-of-bag score: %.2f'%bag_reg.oob_score_)

Train score: 0.57
Test score: 0.58
Out-of-bag score: 0.56


#### Pasting

In [20]:
model = Lasso(fit_intercept=True,alpha=grid_search_lasso.best_params_['alpha'],random_state=0)
bag_reg = BaggingRegressor(model, n_estimators=baggrid.best_params_['n_estimators'],
                           max_samples=baggrid.best_params_['max_samples'],max_features=baggrid.best_params_['max_features'],
                           bootstrap=False, random_state=0)

bag_reg.fit(x_train, y_train)
y_pred = bag_reg.predict(x_test)

In [21]:
print('Train score: %.4f'%bag_reg.score(x_train, y_train))
print('Test score: %.4f'%bag_reg.score(x_test, y_test))

Train score: 0.5731
Test score: 0.5798


### 2. Apply any two models with AdaBoost boosting.

#### Applying AdaBoost to ridge

In [22]:
from sklearn.ensemble import AdaBoostRegressor

param_grid = {'n_estimators': [100,200,300,400,500] ,
              'learning_rate': [0.001,0.01,0.1,1]}


grid_ad = GridSearchCV(AdaBoostRegressor(Ridge(alpha=grid_search_ridge.best_params_['alpha']),random_state=0), param_grid)
grid_ad.fit(x_train, y_train)

print(' best parametes',grid_ad.best_params_)


 best parametes {'learning_rate': 0.001, 'n_estimators': 300}


In [23]:
model = AdaBoostRegressor(Ridge(alpha=grid_search_ridge.best_params_['alpha'],random_state=0),
                                learning_rate= grid_ad.best_params_['learning_rate'],n_estimators=grid_ad.best_params_['n_estimators'])
model.fit(x_train,y_train)
print('Train score: %.4f'%model.score(x_train, y_train))
print('Test score: %.4f'%model.score(x_test, y_test))

Train score: 0.7655
Test score: 0.7536


### Decision Tree Ada Boost Regressor

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
from sklearn.tree import DecisionTreeRegressor
params ={'criterion':['mse'],
    'max_depth':[6,8,10,12,14],
    'min_samples_split':[3,4,5,6],
    'min_samples_leaf':[10,20,30,40,50],
    'ccp_alpha':np.linspace(0.0,1,5) }
grid_search_dt = GridSearchCV(DecisionTreeRegressor(random_state=0),params,cv=5)
grid_search_dt.fit(x_train,y_train)
print("Best parameters: {}".format(grid_search_dt.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search_dt.best_score_))

Best parameters: {'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': 8, 'min_samples_leaf': 20, 'min_samples_split': 3}
Best cross-validation score: 0.7499


In [26]:
from sklearn.ensemble import AdaBoostRegressor

param_grid = {'n_estimators': [100,200,300,400,500] ,
              'learning_rate': [0.001,0.01,0.1,1]}


grid_ad = GridSearchCV( AdaBoostRegressor (DecisionTreeRegressor(ccp_alpha=grid_search_dt.best_params_['ccp_alpha'],
                                          criterion=grid_search_dt.best_params_['criterion'],
                                          max_depth=grid_search_dt.best_params_['max_depth'],
                                          min_samples_leaf=grid_search_dt.best_params_['min_samples_leaf'],
                                          min_samples_split=grid_search_dt.best_params_['min_samples_split'],
                                          random_state=0)), param_grid)
grid_ad.fit(x_train, y_train)

print(' best parametes',grid_ad.best_params_)


 best parametes {'learning_rate': 0.01, 'n_estimators': 400}


In [27]:
model = AdaBoostRegressor(DecisionTreeRegressor(ccp_alpha=grid_search_dt.best_params_['ccp_alpha'],
                                          criterion=grid_search_dt.best_params_['criterion'],
                                          max_depth=grid_search_dt.best_params_['max_depth'],
                                          min_samples_leaf=grid_search_dt.best_params_['min_samples_leaf'],
                                          min_samples_split=grid_search_dt.best_params_['min_samples_split'],
                                          random_state=0),
                                learning_rate= grid_ad.best_params_['learning_rate'],n_estimators=grid_ad.best_params_['n_estimators'])
model.fit(x_train,y_train)
print('Train score: %.4f'%model.score(x_train, y_train))
print('Test score: %.4f'%model.score(x_test, y_test))

Train score: 0.8705
Test score: 0.8132


#### Ada Boost Regressor worked best on the Decision Tree Regrssor the Test score had a formidable increase from 0.74 to 0.81

In [28]:
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [29]:
pred= model.predict(x_test)
print(f'\n Mean Squared Error: {(mean_squared_error(y_test, pred))} \n R2 Score: {r2_score(y_test,pred)}\n Mean Absolute Error: {mean_absolute_error(y_test,pred)}')



 Mean Squared Error: 3.300310576464986 
 R2 Score: 0.8131562283610335
 Mean Absolute Error: 1.394905078302232


### 3. Apply gradient boosting.

#### Apply Gradient Boosting Regressor

In [30]:
from  sklearn.ensemble import GradientBoostingRegressor
param_grid = {'max_depth':[1, 2, 5], 
              'n_estimators':[200, 500, 1000], 
              'learning_rate':[0.001, 0.01, 0.1] }


gb_clfgb= GradientBoostingRegressor(random_state=0)
grid_gb = GridSearchCV(gb_clfgb, param_grid, cv = 5,n_jobs=-1 )
grid_gb.fit(x_train, y_train)

print(" Best parameters: {}".format(grid_gb.best_params_))
#print(" Best score: {}".format(grid_gb.best_score_))

 Best parameters: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200}


In [31]:
print('Train score: %.4f'%grid_gb.score(x_train, y_train))
print('Test score: %.4f'%grid_gb.score(x_test, y_test))

Train score: 0.8712
Test score: 0.8347


In [32]:
pred= grid_gb.predict(x_test)
print(f'\n Mean Squared Error: {(mean_squared_error(y_test, pred))} \n R2 Score: {r2_score(y_test,pred)}\n Mean Absolute Error: {mean_absolute_error(y_test,pred)}')



 Mean Squared Error: 2.9201505058982495 
 R2 Score: 0.8346786092901964
 Mean Absolute Error: 1.2562085900647555


### 4. Apply PCA on data and then apply all the models in project 1 again on the post PCA data. Compare your results with results in project 1. You don't need to apply all the models twice. Just copy the result from project 1, prepare a similar table for all the models after PCA and compare. Does PCA help in getting better results?

In [33]:
from sklearn.decomposition import PCA

In [34]:
pca = PCA(n_components = 0.95)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [35]:
x_train_pca =pca.inverse_transform(x_train_pca)
x_test_pca =pca.inverse_transform(x_test_pca)

### KNN Regressor with PCA

In [36]:
from sklearn.neighbors import KNeighborsRegressor

In [37]:
knn_reg = KNeighborsRegressor()

In [38]:
params = {'n_neighbors':range(1,20,1)}
grid_search = GridSearchCV(knn_reg,params,cv=5)
grid_search.fit(x_train_pca, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Best parameters: {'n_neighbors': 6}
Best cross-validation score: 0.2979


In [39]:
print('KNN Regressor Train Score: ',grid_search.score(x_train_pca,y_train))
print('KNN Regressor Test Score: ',grid_search.score(x_test_pca,y_test))

KNN Regressor Train Score:  0.5143996281943496
KNN Regressor Test Score:  0.33915964853506686


In [40]:

full_table = [['KNN Regressor: ',grid_search.best_params_,grid_search.score(x_train_pca,y_train),grid_search.score(x_test_pca,y_test),
               r2_score(y_test,grid_search.predict(x_test_pca))]]

### Linear Regressor with PCA

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [42]:
linregrr=LinearRegression()
linregrr.fit(x_train_pca, y_train)

print(' Training score :',linregrr.score(x_train_pca, y_train))
print(' Testing score  :',linregrr.score(x_test_pca, y_test))
pred= linregrr.predict(x_test_pca)
print(f'\n Mean Squared Error: {np.sqrt(mean_squared_error(y_test, pred))} \n R2 Score: {r2_score(y_test,pred)}\n Mean Absolute Error: {mean_absolute_error(y_test,pred)}')


 Training score : 0.747713340597516
 Testing score  : 0.738015437738011

 Mean Squared Error: 2.1511759966633726 
 R2 Score: 0.738015437738011
 Mean Absolute Error: 1.710015625434432


In [43]:

full_table += [['Linear Regressor: ',' ',linregrr.score(x_train_pca,y_train),linregrr.score(x_test_pca,y_test),
               r2_score(y_test,linregrr.predict(x_test_pca))]]

### Ridge with PCA

In [44]:
lambdas=np.linspace(13,20,100)
params={'alpha':lambdas}
model=Ridge(fit_intercept=True, random_state=0)
grid_search = GridSearchCV(model,params,cv=5)
grid_search.fit(x_train_pca, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Best parameters: {'alpha': 15.969696969696969}
Best cross-validation score: 0.7324


In [45]:
print('Ridge Train Score: ',grid_search.score(x_train_pca,y_train))
print('Ridge Test Score: ',grid_search.score(x_test_pca,y_test))

Ridge Train Score:  0.7478852973290508
Ridge Test Score:  0.7376315713785951


In [46]:

full_table += [['Ridge Regressor: ',grid_search.best_params_,grid_search.score(x_train_pca,y_train),grid_search.score(x_test_pca,y_test),
               r2_score(y_test,grid_search.predict(x_test_pca))]]

In [47]:
full_table

[['KNN Regressor: ',
  {'n_neighbors': 6},
  0.5143996281943496,
  0.33915964853506686,
  0.33915964853506686],
 ['Linear Regressor: ',
  ' ',
  0.747713340597516,
  0.738015437738011,
  0.738015437738011],
 ['Ridge Regressor: ',
  {'alpha': 15.969696969696969},
  0.7478852973290508,
  0.7376315713785951,
  0.7376315713785951]]

### Lasso with PCA

In [48]:
lambdas=np.linspace(0.1,1,100)
params={'alpha':lambdas}
model=Lasso(fit_intercept=True,random_state=0)
grid_search = GridSearchCV(model,params,cv=5)
grid_search.fit(x_train_pca, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Best parameters: {'alpha': 0.1}
Best cross-validation score: 0.7356


In [49]:
print('Lasso Train Score: ',grid_search.score(x_train_pca,y_train))
print('Lasso Test Score: ',grid_search.score(x_test_pca,y_test))

Lasso Train Score:  0.7392664464592089
Lasso Test Score:  0.7377555442840982


In [50]:

full_table += [['Lasso Regressor: ',grid_search.best_params_,grid_search.score(x_train_pca,y_train),grid_search.score(x_test_pca,y_test),
               r2_score(y_test,grid_search.predict(x_test_pca))]]

### Polynomoial Regression with PCA

##### Fit the Polynomial regression with degree 2

In [51]:
from  sklearn.preprocessing  import PolynomialFeatures

poly = PolynomialFeatures(degree = 2)
x_train_poly = poly.fit_transform(x_train_pca)
x_test_poly = poly.transform(x_test_pca)

poly_reg = LinearRegression()
poly_reg.fit(x_train_poly, y_train)

print(poly_reg.score(x_train_poly, y_train))
print(poly_reg.score(x_test_poly, y_test))

0.8780344161516338
-2.0772917437739646e+23


##### Compare Polynomial Regression degree 2 with 3 and 4

In [52]:
best_score= poly_reg.score(x_test_poly, y_test)

In [53]:
lambdas = [3,4]

In [54]:
best_parameters = {}
for degree in lambdas:
    poly = PolynomialFeatures(degree = degree)
    x_train_poly = poly.fit_transform(x_train_pca)
    x_test_poly = poly.transform(x_test_pca)
    poly_reg = LinearRegression()
    poly_reg.fit(x_train_poly, y_train)
    score = poly_reg.score(x_test_poly, y_test)
    if score > best_score:
        best_score = score
        best_parameters = {'degree': degree}

print("Best score: {:.2f}".format(best_score))
print("Best parameters: {}".format(best_parameters))
    

Best score: -33.95
Best parameters: {'degree': 4}


##### Fit the model with the best degree

In [55]:
from  sklearn.preprocessing  import PolynomialFeatures

poly = PolynomialFeatures(degree = best_parameters['degree'])
x_train_poly = poly.fit_transform(x_train_pca)
x_test_poly = poly.transform(x_test_pca)

poly_reg = LinearRegression()
poly_reg.fit(x_train_poly, y_train)

print(poly_reg.score(x_train_poly, y_train))
print(poly_reg.score(x_test_poly, y_test))

1.0
-33.94963851107548


#### Model is over-fitting

In [56]:

full_table += [['Polynomial Regressor: ',best_parameters,poly_reg.score(x_train_poly,y_train),poly_reg.score(x_test_poly,y_test),
               r2_score(y_test,poly_reg.predict(x_test_poly))]]

### LinearSVR with PCA

In [57]:
from sklearn.svm import LinearSVR
import warnings
warnings.filterwarnings('ignore')


In [58]:
params = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],'max_iter': [1000,10000]}
grid_search = GridSearchCV(LinearSVR(random_state=0),params)
grid_search.fit(x_train_pca,y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Best parameters: {'C': 10, 'max_iter': 1000}
Best cross-validation score: 0.7241


In [59]:
print('LinearSVR Train Score: ',grid_search.score(x_train_pca,y_train))
print('LinearSVR Test Score: ',grid_search.score(x_test_pca,y_test))

LinearSVR Train Score:  0.737708626428641
LinearSVR Test Score:  0.7316131972302815


In [60]:

full_table += [['LinearSVR Regressor: ',grid_search.best_params_,grid_search.score(x_train_pca,y_train),grid_search.score(x_test_pca,y_test),
               r2_score(y_test,grid_search.predict(x_test_pca))]]

### SVR w/ Kernel : 'rbf'

In [62]:
from sklearn.svm import SVR
params = {'C':[1,10,100,1000,5000],'epsilon':[1,1.5,2,5,10],'kernel':['rbf']}
grid_search = GridSearchCV(SVR(),params)
grid_search.fit(x_train_pca,y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Best parameters: {'C': 10, 'epsilon': 1, 'kernel': 'rbf'}
Best cross-validation score: 0.6923


In [63]:
print('SVM Regressor w/ rbf Train Score: ',grid_search.score(x_train_pca,y_train))
print('SVM Regressor w/ rbf Test Score ',grid_search.score(x_test_pca,y_test))

SVM Regressor w/ rbf Train Score:  0.8736915806799034
SVM Regressor w/ rbf Test Score  0.7091449272532566


In [64]:

full_table += [['SVM Regressor w/ rbf: ',grid_search.best_params_,grid_search.score(x_train_pca,y_train),grid_search.score(x_test_pca,y_test),
               r2_score(y_test,grid_search.predict(x_test_pca))]]

### SVR w/ Kernel : 'poly'

In [66]:
params = {'C':[1,10,100,1000,5000],'epsilon':[0.1,1,1.5,2],'kernel':['poly']}
grid_search = GridSearchCV(SVR(),params)
grid_search.fit(x_train_pca,y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Best parameters: {'C': 10, 'epsilon': 1.5, 'kernel': 'poly'}
Best cross-validation score: 0.5859


In [67]:
print('SVM Regressor w/ poly Train Score: ',grid_search.score(x_train_pca,y_train))
print('SVM Regressor w/ poly Test Score: ',grid_search.score(x_test_pca,y_test))

SVM Regressor w/ poly Train Score:  0.8136203037114796
SVM Regressor w/ poly Test Score:  -0.284947991836102


In [68]:

full_table += [['SVM Regressor w/ poly: ',grid_search.best_params_,grid_search.score(x_train_pca,y_train),grid_search.score(x_test_pca,y_test),
               r2_score(y_test,grid_search.predict(x_test_pca))]]

### SVR w/ Kernel : 'linear'

In [69]:
params = {'C':[10,15,20],'epsilon':[1.5,2],'kernel':['linear']}
grid_search = GridSearchCV(SVR(),params)
grid_search.fit(x_train_pca,y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Best parameters: {'C': 15, 'epsilon': 1.5, 'kernel': 'linear'}
Best cross-validation score: 0.7326


In [70]:
print('SVM Regressor w/ linear Train Score: ',grid_search.score(x_train_pca,y_train))
print('SVM Regressor w/ linear Test Score: ',grid_search.score(x_test_pca,y_test))

SVM Regressor w/ linear Train Score:  0.7457977189981425
SVM Regressor w/ linear Test Score:  0.7333964148794789


In [71]:

full_table += [['SVM Regressor w/ linear: ',grid_search.best_params_,grid_search.score(x_train_pca,y_train),grid_search.score(x_test_pca,y_test),
               r2_score(y_test,grid_search.predict(x_test_pca))]]

### Decision Tree Regressor

In [72]:
from sklearn.tree import DecisionTreeRegressor

In [73]:
params ={'criterion':['mse'],
    'max_depth':[6,8,10,12,14],
    'min_samples_split':[3,4,5,6],
    'min_samples_leaf':[10,20,30,40,50],
    'ccp_alpha':np.linspace(0.0,1,5) }
grid_search = GridSearchCV(DecisionTreeRegressor(random_state=0),params,cv=5)
grid_search.fit(x_train_pca,y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Best parameters: {'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': 6, 'min_samples_leaf': 20, 'min_samples_split': 3}
Best cross-validation score: 0.7073


In [74]:
print('Decision Tree Regressor Train Score: ',grid_search.score(x_train,y_train))
print('Decision Tree Regressor Test Score: ',grid_search.score(x_test,y_test))

Decision Tree Regressor Train Score:  0.7190891537428562
Decision Tree Regressor Test Score:  0.718179669206187


In [75]:

full_table += [['Decision Tree Regressor: ',grid_search.best_params_,grid_search.score(x_train_pca,y_train),grid_search.score(x_test_pca,y_test),
               r2_score(y_test,grid_search.predict(x_test_pca))]]

In [76]:
full_table1 = pd.DataFrame(full_table,columns=['model','Hyperparameters','Train score',
                                               'Test score','r2_score'])
full_table1.index = full_table1['model']
full_table1.round(2)
full_table1

,model,Hyperparameters,Train score,Test score,r2_score
model,,,,,
KNN Regressor:,KNN Regressor:,{'n_neighbors': 6},0.514400,0.339160,0.339160
Linear Regressor:,Linear Regressor:,,0.747713,0.738015,0.738015
Ridge Regressor:,Ridge Regressor:,{'alpha': 15.969696969696969},0.747885,0.737632,0.737632
Lasso Regressor:,Lasso Regressor:,{'alpha': 0.1},0.739266,0.737756,0.737756
Polynomial Regressor:,Polynomial Regressor:,{'degree': 4},1.000000,-33.949639,-33.949639
LinearSVR Regressor:,LinearSVR Regressor:,"{'C': 10, 'max_iter': 1000}",0.737709,0.731613,0.731613
SVM Regressor w/ rbf:,SVM Regressor w/ rbf:,"{'C': 10, 'epsilon': 1, 'kernel': 'rbf'}",0.873692,0.709145,0.709145
SVM Regressor w/ poly:,SVM Regressor w/ poly:,"{'C': 10, 'epsilon': 1.5, 'kernel': 'poly'}",0.813620,-0.284948,-0.284948
SVM Regressor w/ linear:,SVM Regressor w/ linear:,"{'C': 15, 'epsilon': 1.5, 'kernel': 'linear'}",0.745798,0.733396,0.733396


### Report Table without PCA

,model,Hyperparameters,Average train score,Average Test score,r2_score
model,,,,,
KNN Regressor:,KNN Regressor:,{'n_neighbors': 5},0.293856,0.296661,0.335935
Linear Regressor:,Linear Regressor:,,0.296661,0.687955,0.753078
Ridge Regressor:,Ridge Regressor:,{'alpha': 15.464646464646465},0.750016,0.699864,0.753646
Lasso Regressor:,Lasso Regressor:,{'alpha': 0.032},0.751155,0.728187,0.754941
Polynomial Regressor:,Polynomial Regressor:,{'degree': 4},-15.051952,-501.839855,-29.360697
LinearSVM Regressor:,LinearSVM Regressor:,"{'C': 10, 'max_iter': 10000}",0.738619,0.684111,0.748209
SVM Regressor w/ rbf:,SVM Regressor w/ rbf:,"{'C': 10, 'epsilon': 1.5, 'kernel': 'rbf'}",0.703479,0.589645,0.715013
SVM Regressor w/ poly:,SVM Regressor w/ poly:,"{'C': 10, 'epsilon': 1, 'kernel': 'poly'}",0.506432,-8.321422,-3.671765
SVM Regressor w/ linear:,SVM Regressor w/ linear:,"{'C': 15, 'epsilon': 1.5, 'kernel': 'linear'}",0.748085,0.696301,0.752905


#### The above two tables show the comparision between all supervised models Run with PCA and without PCA. As we can see the Train scores

#### and r2 scores have dropped for all models. Ridge and Lasso were the best performers in project 1. After PCA the scores for ridge

#### and lasso are still close to theyre previous scores showing that they are pretty robust models.

## 5.Apply deep learning models (MLP) covered in class.

In [77]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
#from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasClassifier, KerasRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

In [78]:
from keras.models import Sequential
from keras.layers import Dense

#### Build a Model

In [79]:
# step 1: build the model
def build_model():
    model = Sequential()
    model.add(Dense(16, input_dim = x_train.shape[1], activation='sigmoid', kernel_initializer = 'zero', name = 'input_layer'))
    model.add(Dense(8, activation='relu', name = 'hidden_layer'))
    model.add(Dense(1, name = 'output_layer'))
    model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mse','mae'])
    return model


#model.summary()

# Compile
#model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mse','mae'])

In [80]:
model = KerasRegressor(build_fn = build_model, verbose = 0,random_state=0)

In [81]:
import warnings
warnings.filterwarnings('ignore')

In [82]:
param_grid = {'batch_size':[10,50,100,150] , 'epochs':[10, 50, 100]}
grid_searchdpl = GridSearchCV(estimator= model, param_grid = param_grid, cv = 5, n_jobs=-1)
grid_searchdpl.fit(x_train,y_train)
print("Best parameters: {}".format(grid_searchdpl.best_params_))
#print("Best cross-validation score: {:.4f}".format(grid_searchdpl.best_score_))

Best parameters: {'batch_size': 10, 'epochs': 100}


In [83]:
tf.random.set_seed(0)

#### Step 3: Model training

In [84]:
model = build_model()
model.fit(x_train, y_train, batch_size=grid_searchdpl.best_params_['batch_size'], epochs=grid_searchdpl.best_params_['epochs'])
model.summary()

Epoch 1/100
176/176 [==============================] - 0s 713us/step - loss: 150.1618 - mse: 150.1618 - mae: 11.5093
Epoch 2/100
176/176 [==============================] - 0s 581us/step - loss: 48.3407 - mse: 48.3407 - mae: 5.7568
Epoch 3/100
176/176 [==============================] - 0s 563us/step - loss: 7.5732 - mse: 7.5732 - mae: 2.1416
Epoch 4/100
176/176 [==============================] - 0s 601us/step - loss: 5.2347 - mse: 5.2347 - mae: 1.8032
Epoch 5/100
176/176 [==============================] - 0s 574us/step - loss: 4.6256 - mse: 4.6256 - mae: 1.6827
Epoch 6/100
176/176 [==============================] - 0s 617us/step - loss: 4.3684 - mse: 4.3684 - mae: 1.6290
Epoch 7/100
176/176 [==============================] - 0s 848us/step - loss: 4.2502 - mse: 4.2502 - mae: 1.5961
Epoch 8/100
176/176 [==============================] - 0s 679us/step - loss: 4.2013 - mse: 4.2013 - mae: 1.5847
Epoch 9/100
176/176 [==============================] - 0s 689us/step - loss: 4.1788 - mse: 4.1788

176/176 [==============================] - 0s 665us/step - loss: 2.9312 - mse: 2.9312 - mae: 1.2882
Epoch 74/100
176/176 [==============================] - 0s 806us/step - loss: 2.9052 - mse: 2.9052 - mae: 1.2806
Epoch 75/100
176/176 [==============================] - 0s 659us/step - loss: 2.8973 - mse: 2.8973 - mae: 1.2762
Epoch 76/100
176/176 [==============================] - 0s 600us/step - loss: 2.8975 - mse: 2.8975 - mae: 1.2817
Epoch 77/100
176/176 [==============================] - 0s 716us/step - loss: 2.8678 - mse: 2.8678 - mae: 1.2725
Epoch 78/100
176/176 [==============================] - 0s 803us/step - loss: 2.8578 - mse: 2.8578 - mae: 1.2735
Epoch 79/100
176/176 [==============================] - 0s 614us/step - loss: 2.8398 - mse: 2.8398 - mae: 1.2681
Epoch 80/100
176/176 [==============================] - 0s 634us/step - loss: 2.8204 - mse: 2.8204 - mae: 1.2604
Epoch 81/100
176/176 [==============================] - 0s 609us/step - loss: 2.8140 - mse: 2.8140 - mae: 1.2

#### Step 4: Model evaluation

In [85]:
# Step 4: Model evaluation

train_loss_and_metrics = model.evaluate(x_train, y_train)
test_loss_and_metrics = model.evaluate(x_test, y_test)

print("Train MSE", train_loss_and_metrics[0])
print("Train MAE", train_loss_and_metrics[2])

print("Test MSE", test_loss_and_metrics[0])
print("Test MAE", test_loss_and_metrics[2])

14/14 [==============================] - 0s 1ms/step - loss: 3.4105 - mse: 3.4105 - mae: 1.4113
Train MSE 2.4384396076202393
Train MAE 1.1720160245895386
Test MSE 3.4105091094970703
Test MAE 1.4112906455993652


In [86]:
y_pred = model.predict(x_test)
r2_score(y_test,y_pred)

0.806917474248434

### Comparing Neural Network scores with the best satistical model : Lasso Regression

In [87]:
from sklearn.metrics import mean_absolute_error , mean_squared_error

In [88]:
pred= grid_search_lasso.predict(x_test)
print(f'\n Mean Squared Error: {mean_squared_error(y_test, pred)} \n R2 Score: {r2_score(y_test,pred)}\n Mean Absolute Error: {mean_absolute_error(y_test,pred)}')



 Mean Squared Error: 4.328015055119251 
 R2 Score: 0.7549737705368076
 Mean Absolute Error: 1.6483784244693234


#### The above results show us that the Artificial nerual network is working better or as good as our best statistical model Lasso Regressor.
#### with the MSE and MAE scores very close to each other.